The convnet used in the original DeepDream release was an Inception model, and in practice Inception is known to produce nice-looking DeepDreams, so we’ll use the Inception V3 model that comes with Keras.

In [1]:
from keras.applications import inception_v3
from keras import backend as K

# to disable the training-specific operations
K.set_learning_phase(0)

# we build the Inception V3 network, but without the convolutional base
# model is loaded with pre-trained ImageNet weights
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


We will now compute the loss: the quantity we’ll seek to maximize during the gradient-ascent process.

In [49]:
# Dictionary mapping layer names to a coefficient quantifying how much the layer’s activation contributes 
# to the loss we’ll seek to maximize. Note that the layer names are hardcoded in the built-in 
# Inception V3 application. We can list all layer names using model.summary().
# Set the contribution that we want specific layers to play.
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 4.,
    'mixed4': 2.,
    'mixed5': 3.5,
}

In [50]:
# Let’s define a tensor that contains the loss: the weighted sum of the L2 norm of 
# the activations of the layers

# Creates a dictionary that maps layer names to layer instances
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Define the loss by adding layer contributions to this scalar variable.
loss = K.variable(0.)

for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output # retrieves layers output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    # Adds the L2 norm of the features of a layer to the loss. 
    # We avoid border artifacts by only involving nonborder pixels in the loss
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

Next, we can set up the gradient-ascent process.

In [82]:
# This tensor holds the generated image: the dream.
dream = model.input
# Computes the gradients of the dream with regard to the loss
grads = K.gradients(loss, dream)[0]
# Normalizes the gradients (important trick)
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# Sets up a Keras function to retrieve the value of the loss and gradients, given an input image
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [84]:
# Some auxiliary functions
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [86]:
# Now the actual DeepDream algorithm
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 40

max_loss = 10.
base_image_path = '/home/ec2-user/stefano_seaside.jpg'

img = preprocess_image(base_image_path)

# Prepares a list of shape tuples defining the different scales at which to run gradient ascent
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
        for dim in original_shape])
    successive_shapes.append(shape)

# Reverses the list of shapes so they’re in increasing order
successive_shapes = successive_shapes[::-1]
# Resizes the Numpy array of the image to the smallest scale
original_img = np.copy(img) 
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='final_dream.png')

Processing image shape (2040, 1363)
...Loss value at 0 : 1.4454918
...Loss value at 1 : 1.9823048
...Loss value at 2 : 2.6629438
...Loss value at 3 : 3.48536
...Loss value at 4 : 4.351623
...Loss value at 5 : 5.2246284
...Loss value at 6 : 6.1000032
...Loss value at 7 : 6.9680915
...Loss value at 8 : 7.829274
...Loss value at 9 : 8.682612
...Loss value at 10 : 9.515408


/usr/lib/python3.4/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  from ipykernel import kernelapp as app


Processing image shape (2857, 1908)
...Loss value at 0 : 3.656387
...Loss value at 1 : 5.1398964
...Loss value at 2 : 6.329356
...Loss value at 3 : 7.4237566
...Loss value at 4 : 8.481479
...Loss value at 5 : 9.478626
Processing image shape (4000, 2672)
...Loss value at 0 : 3.5956554
...Loss value at 1 : 4.9485965
...Loss value at 2 : 6.1025743
...Loss value at 3 : 7.1966476
...Loss value at 4 : 8.280053
...Loss value at 5 : 9.314813
